In [7]:
import torch
import torchvision
import torch.nn
import torch.optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score


# Check for available device (GPU, MPS, or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [8]:
from torch.utils.data import Subset
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import random_split

data_root =  './train_data/'
image_size = (300,300)

transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])

train_dataset = ImageFolder(root=data_root, transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(train_dataset))
test_size = len(train_dataset) - train_size

train_dataset, test_dataset = random_split(train_dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Baseline Model

In [9]:
# Step 3: Build a simple Baseline model.
# You should build a simple/standard neural network, i.e. multilayer perceptron (MLP) trained
# by back-propagation for this step, which helps you have a baseline intuition / idea of the task.
# It is probably easiest to just do this in PyTorch. Don’t use a CNN for this part! The point
# is to get a sense of how well a “vanilla” neural net can do, without convolutions. Because it
# is “dense”, this probably can’t be too large, as it has to be trained in a reasonable amount of
# time. Start small and see how you go.

class MLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
input_size = 300*300*3
hidden_size = 100
num_classes = 3

model = MLP(input_size, hidden_size, num_classes).to(device)

#Train the model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum =0.9)

num_epochs = 10
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        
        #Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        #Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

Epoch [1/10], Loss: 0.9500730037689209
Epoch [2/10], Loss: 1.172192931175232
Epoch [3/10], Loss: 1.1247400045394897
Epoch [4/10], Loss: 0.7774066925048828
Epoch [5/10], Loss: 1.0034619569778442
Epoch [6/10], Loss: 0.9922547340393066
Epoch [7/10], Loss: 1.3390343189239502
Epoch [8/10], Loss: 0.7444821000099182
Epoch [9/10], Loss: 0.7488672137260437
Epoch [10/10], Loss: 0.6170305609703064


In [10]:
# Evaluate the Baseline model.
# Print the train and test accuracy
model.eval()
with torch.no_grad():
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    print(f'Training Accuracy: {100 * correct_train / total_train}%')

    correct_test = 0
    total_test = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 300*300*3).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct_test / total_test}%')

Training Accuracy: 75.25083612040133%
Test Accuracy: 48.60646599777034%
